<div style="font-family: Arial; font-weight: bold; font-size: 2em; color: #FFA500; text-align: center;">
    Python Group Assignment-2
</div>  

<div style="font-family: Arial; font-weight: bold; font-size: 1.3em; color: #FFA500; text-align: center;">
     Import the required libraries and load the data:
</div>   

### Question-1 Load the required libraries and read the dataset 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from scipy import stats
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from scipy.stats import zscore
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster

from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs

In [ ]:
df = pd.read_csv('renttherunway.csv')

### Question- 2  Check the first few samples, shape, info of the data and try to familiarize yourself with different features


In [ ]:
df.sample()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
print('Rows: ', df.shape[0])
print('Columns: ', df.shape[1])

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum().sum()

<div style="font-family: Arial; font-weight: bold; font-size: 1.3em; color: #FFA500; text-align: center;">
     Data cleansing and Exploratory data analysis:
</div>   

### Question- 3  Check if there are any duplicate records in the dataset?

In [ ]:
len(df[df.duplicated()])

### Question- 4 Drop the columns which you think redundant for the analysis.

In [ ]:
df = df.copy()

In [ ]:
df.drop(['item_id','review_text','review_date'], axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
df.shape

### Question- 5 Check the column 'weight', Is there any presence of string data? If yes, remove the string data and convert to float.

In [ ]:
Data = df[['weight']]

In [ ]:
df['weight'] = df['weight'].str[:-3].astype(float)
df['weight'].info()

In [ ]:
df["weight"].sample(5)

### Question-6  Check the unique categories for the column 'rented for' and group 'party: cocktail' category with 'party'.

In [ ]:
unique_categories = df['rented for'].unique()
print("Unique categories for 'rented for':", unique_categories)

In [ ]:
df['rented for'] = df['rented for'].replace('party: cocktail', 'party')

In [ ]:
updated_categories = df['rented for'].unique()
print("Updated unique categories for 'rented for':", updated_categories)

### Question-7 The column 'height' is in feet with a quotation mark, Convert to inches with float datatype. 

In [ ]:
df['height'].sample(10)

In [ ]:
import re
df['height'] = df['height'].astype(str)

def convert_to_inches(height):
    match = re.match(r'(\d+)\'\s*(\d+)?', height)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2)) if match.group(2) else 0
        return (feet * 12) + inches
    else:
        return None

df['height'] = df['height'].apply(convert_to_inches)


In [ ]:
print("Unique heights (after conversion):", df['height'].unique())

### Question-8  Check for missing values in each column of the dataset? If it exists, impute them with appropriate methods. 

In [ ]:
missing_values = df.isnull().sum()

categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

numerical_cols = df.select_dtypes(include=['float', 'int']).columns
for col in numerical_cols:
    df[col].fillna(df[col].mean(), inplace=True)

missing_values_after_imputation = df.isnull().sum()

print(df.head())

In [ ]:
missing_values = df.isnull().sum()*100/len(df)

missing_values

### Question-9  Check the statistical summary for the numerical and categorical columns and write your findings.

In [ ]:
numerical_data = df.describe()
numerical_data

In [ ]:
categorical_summary= df.describe(include=['object'])

print("\nStatistical summary for categorical columns:")
categorical_summary

In [ ]:
numerical_data = df.select_dtypes(include=['number']).columns
categorical_columns = df.select_dtypes(include=['object']).columns
numerical_data = df[numerical_data].describe()
numerical_data = df[['size', 'age']].dropna()

In [ ]:
categorical_summary = {}
for column in categorical_columns:
    categorical_summary[column] = df[column].value_counts()

In [ ]:
numerical_data

In [ ]:
for column, summary in categorical_summary.items():
    summary

### Question-10  Are there outliers present in the column age? If yes, treat them with the appropriate method.

In [ ]:
mean_age = df['age'].mean()
std_age = df['age'].std()

In [ ]:
threshold = 3 * std_age

In [ ]:
outliers = df[(df['age'] - mean_age).abs() > threshold]

In [ ]:
outliers

In [ ]:
median_age = df['age'].median()
df.loc[outliers.index, 'age'] = median_age

In [ ]:
df

### Question-11  Check the distribution of the different categories in the column 'rented for' using appropriate plot. 

In [ ]:
category_counts = df['rented for'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=category_counts.index, y=category_counts.values)
plt.title('Distribution of Categories in the "rented for" Column')
plt.xlabel('Category')
plt.ylabel('Frequency')
plt.show()

<div style="font-family: Arial; font-weight: bold; font-size: 1.3em; color: #FFA500; text-align: center;">
     Data Preparation for model building:
</div>   

### Question-12  Encode the categorical variables in the dataset.

In [ ]:
categorical_columns = []

for column in df.columns:
    if df[column].dtype == 'object':
        categorical_columns.append(column)

print("Categorical columns:", categorical_columns)

In [ ]:
label_encoder = LabelEncoder()

for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])

df.head()

### Question-13  Standardize the data, so that the values are within a particular range.

In [ ]:
scaler = StandardScaler()
standardized_data = scaler.fit_transform(numerical_data)

<div style="font-family: Arial; font-weight: bold; font-size: 1.3em; color: #FFA500; text-align: center;">
     Principal Component Analysis and Clustering:
</div>   

### Question-14   Apply PCA on the above dataset and determine the number of PCA components to be used so that 90-95% of the variance in data is explained by the same.

In [ ]:
missing_values = np.isnan(standardized_data).sum()

infinity_values = np.isinf(standardized_data).sum()

data_max = np.max(standardized_data)

missing_values, infinity_values, data_max

In [ ]:
pca = PCA()
pca.fit(standardized_data)
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = explained_variance_ratio.cumsum()

In [ ]:
n_components = 0
for i, var in enumerate(cumulative_variance):
    if var >= 0.90:
        n_components = i + 1
        break
print(f"Number of components to explain 90% of the variance: {n_components}")

In [ ]:
n_components_95 = 0
for i, var in enumerate(cumulative_variance):
    if var >= 0.95:
        n_components_95 = i + 1 
        break

print(f"Number of components to explain 95% of the variance: {n_components_95}")

### Question-15   Apply K-means clustering and segment the data. (You may use original data or PCA transformed data) 

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(standardized_data)

df['Cluster'] = np.nan
df.loc[numerical_data.index, 'Cluster'] = clusters

print(df[['size', 'age', 'Cluster']].dropna().head())

### Question-15   a. Find the optimal K Value using elbow plot for K Means clustering.

In [ ]:
from tqdm import tqdm

numerical_data = df[['size', 'age']].dropna()

ssd = []
for k in tqdm(range(1, 11)):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(standardized_data)
    ssd.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), ssd, 'bo-', linewidth=2)
plt.title('Elbow Plot for Optimal K Value')
plt.xlabel('Number of Clusters')
plt.ylabel('Sum of Squared Distances')
plt.grid(True)
plt.show()

### Question-15  b. Build a Kmeans clustering model using the obtained optimal K value from the elbow plot.

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=100)

kmeans.fit(pca.transform(standardized_data))

### Question-15  c. Compute silhouette score for evaluating the quality of the K Means clustering technique.

In [ ]:
silhouette_avg = silhouette_score(standardized_data, clusters)

silhouette_avg

### Question-16  Apply Agglomerative clustering and segment the data. (You may use original data or PCA transformed data)

In [ ]:
from sklearn.utils import shuffle
scaled_features_df = pd.DataFrame(standardized_data)
scaled_features_df = shuffle(scaled_features_df, random_state=42).sample(frac=0.1, random_state=42)

In [ ]:
merg = linkage(scaled_features_df, method= 'ward') 
dendrogram(merg, leaf_rotation = 90,) 
plt.show()

In [ ]:
df = pd.DataFrame(index=range(19158))
values = list(range(19158)) 
df['column_name'] = values
agg_clustering = AgglomerativeClustering(n_clusters=3)
agg_clusters = agg_clustering.fit_predict(scaled_features_df)

df['Agg_Cluster'] = agg_clusters

df.head()

### Question-16  a. Find the optimal K Value using dendrogram for Agglomerative clustering. 

In [ ]:
Z = hierarchy.linkage(df, method='ward')
plt.figure(figsize=(10, 5))
dn = hierarchy.dendrogram(Z)
plt.title('Dendrogram for Agglomerative Clustering')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.show()

### Question-16  b. Build a Agglomerative clustering model using the obtained optimal K value observed from dendrogram.

In [ ]:
agg_model = AgglomerativeClustering(n_clusters=3)
agg_clusters = agg_model.fit_predict(scaled_features_df)
plt.figure(figsize=(10, 5))
Z = hierarchy.linkage(df, method='ward')
df = hierarchy.dendrogram(Z)
plt.title('Dendrogram for Agglomerative Clustering')
plt.xlabel('Sample Index')
plt.ylabel('Distance')
plt.show()

### Question-16  c. Compute silhouette score for evaluating the quality of the Agglomerative clustering technique. 

In [ ]:
silhouette_avg = silhouette_score(scaled_features_df, agg_clusters)

silhouette_avg

In [ ]:
scaled_features_df.isna().sum().sum()
np.isinf(scaled_features_df).sum().sum()

In [ ]:
print(type(scaled_features_df))

### Question-17  Perform cluster analysis by doing bivariate analysis between cluster labels and different features and write your conclusion on the results.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5), facecolor='white')

axs[0].scatter(df['size'], df['rating'], c=df['cluster'])
axs[0].set_title('Rating vs Size')
axs[0].set_xlabel('Size')
axs[0].set_ylabel('Rating')

linked = linkage(scaled_features_df, 'ward')
dendrog

features = ['rating', 'size']
df_subset = df[features].dropna()

scaler = StandardScaler()
standardized_data = scaler.fit_transform(data_subset)

cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
cluster_labels = cluster.fit_predict(scaled_features_df)

data_subset['cluster'] = cluster_labels

fig, axs = plt.subplots(1, 2, figsize=(14, 5), facecolor='white')

axs[0].scatter(df['size'], df['rating'], c=df['cluster'])
axs[0].set_title('Rating vs Size')
axs[0].set_xlabel('Size')
axs[0].set_ylabel('Rating')

linked = linkage(scaled_features_df, 'ward')
dendrogram(linked, ax=axs[1], truncate_mode='level', p=5)
axs[1].set_title('Dendrogram')

plt.tight_layout()
fig, axs = plt.subplots(2)
dendrogram(linked, ax=axs[0])
dendrogram(linked, ax=axs[1], truncate_mode='level', p=5)

axs[1].set_title('Dendrogram')

plt.tight_layout()
plt.show()